Processamento de Números de Processos Judiciais.

    Este projeto processa números de processos judiciais a partir de um arquivo Excel, formata os números, verifica a validade e salva os dados processados em um novo arquivo Excel.

    Funcionalidades:
. Formatação de números de processos judiciais para 20 caracteres.
. Verificação de números vazios ou contendo apenas zeros.
. Verificação e classificação de números de processos como NUP (Número Único de Processo) ou Correspondência.
. Leitura de dados de um arquivo Excel.
. Salvamento de dados processados em um arquivo Excel com múltiplas planilhas.
. Filtragem e organização dos dados processados.
    Dependências:
. pandas
. re

    Função "formatar_numero_processo(numero)":
- Formatação de Número de Processo
    Formata um número de processo removendo todos os caracteres não numéricos e ajustando o tamanho para 20 caracteres com preenchimento à esquerda com zeros, se necessário.
  
    Parâmetros:
- numero (str): O número do processo judicial.

    Retorna:
- (str): O número do processo formatado ou uma string vazia se o número não for válido.
    Log:
. Emite um INFO se o número tem menos de 18 caracteres.
. Emite um INFO se o número tem mais de 20 caracteres.

    Função "verificar_vazio_ou_zeros(numero_formatado)":
- Verifica se o número formatado está vazio ou contém apenas zeros.

     Parâmetros:
- numero_formatado (str): O número do processo formatado.

    Retorna:
- (bool): True se o número estiver vazio ou contiver apenas zeros, caso contrário, False.

    Função "verifica_nup_jud(numero_formatado)":
- Verifica se o número formatado é um NUP válido ou uma correspondência.
    Parâmetros:
- numero_formatado (str): O número do processo formatado.

    Retorna:
- (str): 'NUP' se for um NUP válido, 'Correspondencia' se for uma correspondência, 'Invalido' se for inválido.

    Regras:
. Verifica segmentos de data válidos (1950-2025).
. Usa regex para verificar padrões específicos de NUP.

    Função "ler_excel(arquivo_entrada)":
- Lê o arquivo Excel de entrada e retorna um DataFrame.

    Parâmetros:
- arquivo_entrada (str): O caminho para o arquivo Excel de entrada.

    Retorna:
- (DataFrame): O DataFrame com os dados do arquivo Excel, ou None em caso de erro.

    Função "salvar_excel(dfs, arquivo_saida)":
- Salva os DataFrames em um arquivo Excel com múltiplas planilhas, separando e renomeando as colunas desejadas.

    Parâmetros:
- dfs (dict): Dicionário de DataFrames a serem salvos, onde as chaves são os nomes das planilhas.
- arquivo_saida (str): O caminho para o arquivo Excel de saída.

    Retorna:
- None

    Função "processar_dados_excel(arquivo_entrada)":
- Processa o arquivo Excel de entrada e retorna DataFrames processados.

    Parâmetros:
 - arquivo_entrada (str): O caminho para o arquivo Excel de entrada.

    Retorna:
- (dict): Um dicionário com os DataFrames processados:
    'Processos Válidos': DataFrame com os processos válidos.
    'NUP': DataFrame com os processos NUP.
    'Processos Inválidos': DataFrame com os processos inválidos.
     None em caso de erro.

    Contribuição: 
- Contribuições são bem-vindas! Sinta-se à vontade para abrir uma issue ou enviar um pull request.

    Licença:
- Este projeto está licenciado sob a MIT License.



In [ ]:
import pandas as pd
import re
import logging

# Configura o nível de log para INFO
logging.basicConfig(level=logging.INFO)


def formatar_numero_processo(numero):
    """Formata o número do processo para 20 caracteres, se possível."""
    numero_formatado = re.sub(r'\D', '', str(numero))
    tamanho = len(numero_formatado)

    if 18 <= tamanho <= 20:
        return numero_formatado.zfill(20)
    elif tamanho == 17:
        return 'NUP'
    elif tamanho > 20:
        logging.info(f"Tamanho incorreto. Maior que 20 caracteres: {numero_formatado}")
    else:
        logging.info(f"Numero com menos de 18 caracteres: {numero_formatado}")

    return ''


def verificar_vazio_ou_zeros(numero_formatado):
    """Verifica se o número formatado está vazio ou contém apenas zeros."""
    return not numero_formatado or numero_formatado == '0' * len(numero_formatado)


def verifica_nup_jud(numero_formatado):
    """Verifica se o número formatado é um NUP válido ou uma Correspondência."""
    if not numero_formatado:
        return 'Invalido'

    pattern = r'^0*([45]\d*)$'
    regex = re.compile(pattern)
    numero = str(numero_formatado)

    def segmento_de_data_valido(segmento):
        return 1950 <= int(segmento) <= 2025

    if len(numero) > 2 and segmento_de_data_valido(numero[-6:-2]) and regex.match(numero):
        return 'NUP'
    elif len(numero) > 7 and segmento_de_data_valido(numero[-11:-7]):
        return 'Correspondencia'

    return 'Invalido'


def ler_excel(arquivo_entrada):
    """Lê o arquivo Excel de entrada e retorna um DataFrame."""
    try:
        return pd.read_excel(arquivo_entrada)
    except Exception as e:
        logging.error(f"Erro ao ler o arquivo Excel: {e}")
        return None


def salvar_excel(dfs, arquivo_saida):
    """Salva os DataFrames em um arquivo Excel com múltiplas planilhas."""
    try:
        with pd.ExcelWriter(arquivo_saida) as writer:
            for nome, df in dfs.items():
                # Renomear colunas antes de salvar
                df = df.rename(columns={
                    'Numero do Processo': 'PROCESSO',
                    'Valor Uniao': 'VALOR ECONOMICO'
                })

                # Selecionar apenas as colunas desejadas
                df = df[['PROCESSO', 'VALOR ECONOMICO']]

                df.to_excel(writer, sheet_name=nome, index=False)
        logging.info(f'Arquivo {arquivo_saida} salvo com sucesso!')
    except Exception as e:
        logging.error(f"Erro ao salvar o arquivo Excel: {e}")


def processar_dados_excel(arquivo_entrada):
    """Processa o arquivo Excel de entrada e retorna DataFrames processados."""
    df = ler_excel(arquivo_entrada)
    if df is None:
        return None

    if 'Numero do Processo' not in df.columns or 'Data de Saída NECAP' not in df.columns:
        logging.error("As colunas 'Numero do Processo' ou 'Data de Saída NECAP' não foram encontradas no arquivo Excel.")
        return None

    df['Numero do Processo'] = df['Numero do Processo'].apply(formatar_numero_processo)
    df = df[~df['Numero do Processo'].apply(verificar_vazio_ou_zeros) & (df['Numero do Processo'] != '')]

    df_5_zeros = df[df['Numero do Processo'].str.startswith('00000')].copy()
    df_outros = df[~df['Numero do Processo'].str.startswith('00000')]

    df_5_zeros['Verificacao'] = df_5_zeros['Numero do Processo'].apply(verifica_nup_jud)
    df_nup = df_5_zeros[df_5_zeros['Verificacao'] == 'NUP']
    df_correspondencia = df_5_zeros[df_5_zeros['Verificacao'] == 'Correspondencia']
    df_invalidos = df_5_zeros[df_5_zeros['Verificacao'] == 'Invalido']

    df_validos = pd.concat([df_outros, df_correspondencia])
    df_validos['Data de Saída NECAP'] = pd.to_datetime(df_validos['Data de Saída NECAP'], errors='coerce')
    df_validos = df_validos.sort_values(by='Data de Saída NECAP').drop_duplicates(subset=['Numero do Processo'],
                                                                                  keep='last')

    return {
        'Processos Válidos': df_validos.drop(columns='Verificacao', errors='ignore'),
        'NUP': df_nup.drop(columns='Verificacao', errors='ignore'),
        'Processos Inválidos': df_invalidos.drop(columns='Verificacao', errors='ignore')
    }



In [ ]:
Requisitos:
. Python 3.x
. Pandas
. re (Regular Expressions)
. logging